In [ ]:
import copy
import dataclasses
import os
import time
import pathlib
import itertools
import multiprocessing
import scipy
import numpy as np
import pandas as pd
import pickle
import gzip
import threading
import queue
import pytz
import traceback
from datetime import datetime
from tqdm.auto import tqdm, trange
from typing import Any

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
!nvidia-smi

In [ ]:
def benchmark(f, *, f_setup=None, min_repeat: int, min_secs: float, tqdm_kwargs: dict | None=None) -> np.ndarray:
    latency = []
    
    # First run, ignore min_secs
    if f_setup is not None:
        f_setup()
    st = time.perf_counter_ns()
    f()
    ed = time.perf_counter_ns()
    latency.append((ed-st)/1e9)
    
    # Subsequent runs, until reaching both min_repeat and min_secs
    min_nanos = int(min_secs * 1e9)
    start_nanos = time.perf_counter_ns()
    while True:
        now_nanos = time.perf_counter_ns()
        if len(latency) > min_repeat and now_nanos - start_nanos > min_nanos:
            break
        if f_setup is not None:
            f_setup()
        st = time.perf_counter_ns()
        f()
        ed = time.perf_counter_ns()
        latency.append((ed-st)/1e9)
    return np.array(latency)

def tail_mean(xs, skip=0.2):
    return xs[int(len(xs) * skip):].mean()

In [ ]:
import torch
torch.set_grad_enabled(False)

In [ ]:
nd_list = list(itertools.chain(itertools.product([12, 16, 32], [64]), itertools.product([32, 40, 56, 72, 96], [128])))
seqlen_list = [10, 20, 50, 100, 200, 500, 1000, 2000, 5000]
bs_list = list(itertools.chain(range(1, 8), range(8, 16, 2), range(16, 32, 4), range(32, 64, 8), range(64, 128, 16), [128]))

In [ ]:
print(nd_list)
print(seqlen_list)
print(bs_list)

In [ ]:
def benchmark_dense(out, nd_list, seqlen_list, bs_list):
    seqlen_list = [1] + seqlen_list
    total = len(list(itertools.product(nd_list, seqlen_list, bs_list)))
    pbar = tqdm(total=total)
    for (n, d), seqlen in reversed(list(itertools.product(nd_list, seqlen_list))):
        h = n * d
        maxbs = max(bs_list)
        cache = torch.empty(int(256e6 // 4), dtype=torch.int, device="cuda:0")
        X = torch.rand((maxbs, seqlen, h), dtype=torch.bfloat16, device="cuda:0")
        W = torch.rand((h, h), dtype=torch.bfloat16, device="cuda:0")
        torch.cuda.synchronize()
        for bs in reversed(bs_list):
            pbar.set_postfix(n=n, h=h, d=d, seqlen=seqlen, bs=bs)
            def run():
                torch.matmul(X[:bs], W)
                torch.cuda.synchronize()
            def clear_cache():
                cache.zero_()
                torch.cuda.synchronize()
            latency = benchmark(run, f_setup=clear_cache, min_repeat=20, min_secs=2)
            l = tail_mean(latency)
            out.append({
                "n": n,
                "d": d,
                "seqlen": seqlen,
                "bs": bs,
                "latency": l
            })
            pbar.update()
        del cache, X, W
        torch.cuda.empty_cache()
    pbar.close()

In [ ]:
def benchmark_qk_init(out, nd_list, seqlen_list, bs_list):
    total = len(list(itertools.product(nd_list, seqlen_list, bs_list)))
    pbar = tqdm(total=total)
    for (n, d), seqlen in reversed(list(itertools.product(nd_list, seqlen_list))):
        h = n * d
        try:
            maxbs = max(b for b in bs_list if b*n*seqlen*d*2*2+b*n*seqlen**2*2 < 80e9)
        except ValueError:
            pbar.update(len(bs_list))
            continue
        cache = torch.empty(int(256e6 // 4), dtype=torch.int, device="cuda:0")
        Qmax = torch.rand((maxbs, n, seqlen, d), dtype=torch.bfloat16, device="cuda:0")
        Kmax = torch.rand((maxbs, n, seqlen, d), dtype=torch.bfloat16, device="cuda:0")
        torch.cuda.synchronize()
        for bs in reversed(bs_list):
            pbar.set_postfix(n=n, h=h, d=d, seqlen=seqlen, bs=bs)
            if bs > maxbs:
                pbar.update()
                continue
            Q = Qmax[:bs]
            K = Kmax[:bs]
            def run():
                torch.bmm(Q.view(bs * n, seqlen, d), K.view(bs * n, seqlen, d).transpose(1, 2))
                torch.cuda.synchronize()
            def clear_cache():
                cache.zero_()
                torch.cuda.synchronize()
            latency = benchmark(run, f_setup=clear_cache, min_repeat=20, min_secs=2)
            l = tail_mean(latency)
            out.append({
                "n": n,
                "d": d,
                "seqlen": seqlen,
                "bs": bs,
                "latency": l
            })
            pbar.update()
        del cache, Q, K, Qmax, Kmax
        torch.cuda.empty_cache()
    pbar.close()

In [ ]:
def benchmark_qk_ar(out, nd_list, seqlen_list, bs_list):
    total = len(list(itertools.product(nd_list, seqlen_list, bs_list)))
    pbar = tqdm(total=total)
    for (n, d), seqlen in reversed(list(itertools.product(nd_list, seqlen_list))):
        h = n * d
        try:
            maxbs = max(b for b in bs_list if b*n*(1+seqlen)*d*2+b*n*seqlen*2 < 80e9)
        except ValueError:
            pbar.update(len(bs_list))
            continue
        cache = torch.empty(int(256e6 // 4), dtype=torch.int, device="cuda:0")
        Qmax = torch.rand((maxbs, n, 1, d), dtype=torch.bfloat16, device="cuda:0")
        Kmax = torch.rand((maxbs, n, seqlen, d), dtype=torch.bfloat16, device="cuda:0")
        torch.cuda.synchronize()
        for bs in reversed(bs_list):
            pbar.set_postfix(n=n, h=h, d=d, seqlen=seqlen, bs=bs)
            if bs > maxbs:
                pbar.update()
                continue
            Q = Qmax[:bs]
            K = Kmax[:bs]
            def run():
                torch.bmm(Q.view(bs * n, 1, d), K.view(bs * n, seqlen, d).transpose(1, 2))
                torch.cuda.synchronize()
            def clear_cache():
                cache.zero_()
                torch.cuda.synchronize()
            latency = benchmark(run, f_setup=clear_cache, min_repeat=20, min_secs=2)
            l = tail_mean(latency)
            out.append({
                "n": n,
                "d": d,
                "seqlen": seqlen,
                "bs": bs,
                "latency": l
            })
            pbar.update()
        del cache, Q, K, Qmax, Kmax
        torch.cuda.empty_cache()
    pbar.close()

In [ ]:
data = {}

In [ ]:
db = []
benchmark_qk_init(db, nd_list, seqlen_list, bs_list)
data["qk_init"] = db

In [ ]:
db = []
benchmark_qk_ar(db, nd_list, seqlen_list, bs_list)
data["qk_ar"] = db

In [ ]:
db = []
benchmark_dense(db, nd_list, seqlen_list, bs_list)
data["dense"] = db

In [ ]:
with gzip.open("data/20230516-transformer-batching.pkl.gz", "wb") as f:
    pickle.dump(data, f)

In [ ]:
df_dense = (
    pd.DataFrame.from_dict(data["dense"])
    .assign(h=lambda x: x["n"] * x["d"])
    .assign(flop=lambda x: (x["bs"] * x["seqlen"] * x["h"]**2) * 2)
    .assign(io=lambda x: (x["bs"]*x["seqlen"]*x["h"]*2 + x["h"]**2) * 2)
    .assign(intensity=lambda x: x["flop"] / x["io"])
    .assign(throughput=lambda x: x["bs"]*x["seqlen"] / x["latency"])
    .assign(series="dense")
)
df_qk_init = (
    pd.DataFrame.from_dict(data["qk_init"])
    .assign(h=lambda x: x["n"] * x["d"])
    .assign(flop=lambda x: (x["bs"]*x["n"]*x["d"]*x["seqlen"]**2) * 2)
    .assign(io=lambda x: (x["bs"]*x["n"]*(x["seqlen"]*x["d"]*2 + x["seqlen"]**2)) * 2)
    .assign(intensity=lambda x: x["flop"] / x["io"])
    .assign(throughput=lambda x: x["bs"]*x["seqlen"] / x["latency"])
    .assign(series="qk_init")
)
df_qk_ar = (
    pd.DataFrame.from_dict(data["qk_ar"])
    .assign(h=lambda x: x["n"] * x["d"])
    .assign(flop=lambda x: (x["bs"]*x["n"]*x["d"]*x["seqlen"]) * 2)
    .assign(io=lambda x: (x["bs"]*x["n"]*(x["d"] + x["seqlen"]*x["d"] + x["seqlen"])) * 2)
    .assign(intensity=lambda x: x["flop"] / x["io"])
    .assign(throughput=lambda x: x["bs"] / x["latency"])
    .assign(series="qk_ar")
)
pd.concat([df_dense, df_qk_init, df_qk_ar]).to_csv("data/transformer-batching-microbenchmarks.csv", index=False)

In [ ]:
import transformers

In [ ]:
def _gen_opt_cfg(n_layers: int, d_model: int, n_heads: int, **kwargs) -> transformers.OPTConfig:
    return transformers.OPTConfig(
        num_hidden_layers=n_layers,
        hidden_size=d_model,
        ffn_dim=d_model*4,
        num_attention_heads=n_heads,
        **kwargs
    )
optcfg = {
    # https://arxiv.org/pdf/2205.01068.pdf   Table 2.1
    "125m": _gen_opt_cfg(12,   768, 12),
    "350m": _gen_opt_cfg(24,  1024, 16),
    "760m": _gen_opt_cfg(24,  1536, 16),
    "1.3b": _gen_opt_cfg(24,  2048, 32),
    "2.7b": _gen_opt_cfg(32,  2560, 32),
    "6.7b": _gen_opt_cfg(32,  4096, 32),
     "13b": _gen_opt_cfg(40,  5120, 40),
     "13b_1layer": _gen_opt_cfg(1,  5120, 40),
     "30b": _gen_opt_cfg(48,  7168, 56),
     "66b": _gen_opt_cfg(64,  9216, 72),
    "175b": _gen_opt_cfg(96, 12288, 96),
    "175b_1layer": _gen_opt_cfg(1, 12288, 96),
}

In [ ]:
def greedy_sample_one(model, input_ids, attention_mask=None, past_key_values=None):
    bs, tgt_len = input_ids.shape
    if past_key_values is not None:
        _bs, _num_heads, src_len, _head_dims = past_key_values[0][0].shape
        assert bs == _bs
    else:
        src_len = 0
    if attention_mask is None:
        attention_mask = torch.ones((bs, src_len + tgt_len), device=model.device)
    ret = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        past_key_values=past_key_values,
        use_cache=True, output_hidden_states=False, return_dict=True,
    )
    return ret

def time_greedy_generate(model, input_ids, new_tokens):
    ts = []
    output = input_ids
    past_key_values = None
    cache = torch.empty(int(256e6 // 4), dtype=torch.int, device=model.device)
    attention_mask = torch.ones(input_ids.shape, device=model.device)    
    for _ in range(new_tokens):
        cache.zero_()
        torch.cuda.synchronize()
        st = time.perf_counter_ns()
        
        ret = greedy_sample_one(model, input_ids, attention_mask, past_key_values)
        input_ids = torch.argmax(ret.logits[:, -1, :], axis=-1)[:, None]
        output = torch.cat([output, input_ids], axis=1)
        past_key_values = ret.past_key_values
        attention_mask = torch.cat([attention_mask, attention_mask.new_ones((attention_mask.shape[0], 1))], dim=-1)
        
        torch.cuda.synchronize()
        ed = time.perf_counter_ns()
        ts.append((ed-st)/1e9)
    return np.array(ts)

In [ ]:
opt_config = optcfg["6.7b"]

torch.set_default_dtype(torch.bfloat16)
with transformers.modeling_utils.no_init_weights():
    model = transformers.models.opt.OPTForCausalLM(opt_config).to("cuda")
torch.set_default_dtype(torch.float32)

In [ ]:
db = {}
input_tokens = 200
new_tokens = 500
for bs in tqdm(list(itertools.chain(range(1, 8), range(8, 16, 2), [16]))):
    x = torch.randint(1000, 10000, (bs, input_tokens), device=model.device)
    stack = []
    for _ in range(10):
        l = time_greedy_generate(model, x, new_tokens=new_tokens)
        stack.append(l)
    db[bs] = np.median(np.stack(stack), axis=0)
    del x
    torch.cuda.empty_cache()
del model
torch.cuda.empty_cache()

with gzip.open("data/20230516-e2e-text-generation-batch.pkl.gz", "wb") as f:
    pickle.dump(db, f)